In [1]:
import numpy as np

def confidence_noise_determination(img, D, m, c, v, beta):
    """
    Parameters:
    M: deterministic mechanism
    D: data distribution
    m: sampling complexity
    c, v, beta: constants

    Returns:
    Gaussian covariance matrix ΣB
    """

    # Step 2: Independently generate data
    y = img

    # Step 6: Calculate empirical mean and empirical covariance estimation
    mu_hat = np.mean(img, axis=(0, 3))
    empirical_cov = np.var(img, axis=(0, 3))

    # Step 7: Apply singular value decomposition (SVD)
    U, Lambda, UT = np.linalg.svd(empirical_cov)
    
    # Step 8: Determine the maximal index j0
    j0 = np.argmin([lambd for lambd in Lambda if lambd > c])
    
    # Step 9: Check the condition
    condition = np.min([np.abs(Lambda[j] - lambd)for j in range(1, j0 + 1) for lambd in Lambda]) > r * np.sqrt(len(Lambda) * c) + 2 * c
    
    if condition:
        # Step 10 & 11: Determine the j-th element of a diagonal matrix ΛB
        sqrt_sum = sum(np.sqrt(Lambda[j] + 10 * c * v/ beta) for j in range(len(Lambda)))
        Lambda_B = [2 * v / np.sqrt(Lambda[j] + 10 * c * v/ beta) * sqrt_sum for j in range(len(Lambda))]
        ΛB = np.diag(Lambda_B)

        # Step 13: Determine the Gaussian noise covariance
        ΣB = U @ np.linalg.inv(ΛB) @ UT

    else:
        # Step 15: Determine the Gaussian noise covariance
        ΣB = (sum(Lambda) + len(Lambda) * c) / (2 * v) * np.eye(len(Lambda))

    return ΣB


In [2]:
import os
from PIL import Image

In [3]:
img_dir = '/data/local/qipan/exp_celeba/fid_samples_larger_batch_2/images_exp_10/smile/ckpt_300000'
img = [Image.open(os.path.join(img_dir, f)) for f in os.listdir(img_dir)]
img = np.array([np.array(i) for i in img])

In [4]:
img.shape

(300, 64, 64, 3)

In [5]:
mu_hat = np.mean(img, axis=(0, 3))
empirical_cov = np.var(img, axis=(0, 3))

In [6]:
mu_hat.shape

(64, 64)

In [7]:
U, Lambda, UT = np.linalg.svd(empirical_cov)

In [8]:
j0 = np.argmin([lambd for lambd in Lambda if lambd > 20])
condition = np.min([np.abs(Lambda[j] - lambd)for j in range(j0 + 1) for lambd in Lambda]) > 1500 * np.sqrt(len(Lambda) * 20) + 2 * 20


In [9]:
condition

False

In [10]:
Lambda

array([3.16958464e+05, 1.82707017e+04, 1.52293405e+04, 8.56201268e+03,
       4.06805894e+03, 3.35912451e+03, 2.90601713e+03, 2.58754395e+03,
       2.07750040e+03, 1.87754155e+03, 1.80099727e+03, 1.59036000e+03,
       1.38040969e+03, 1.31585009e+03, 1.25248656e+03, 1.18059484e+03,
       1.08709934e+03, 1.02204413e+03, 9.49837709e+02, 8.42380330e+02,
       8.33120373e+02, 7.76822059e+02, 7.51962344e+02, 6.74389237e+02,
       6.48737302e+02, 6.07517126e+02, 5.76689662e+02, 5.56027967e+02,
       5.37717156e+02, 4.98210727e+02, 4.76233509e+02, 4.67745595e+02,
       4.25451191e+02, 3.95458731e+02, 3.69005804e+02, 3.44696076e+02,
       3.20466810e+02, 3.06256695e+02, 2.92507743e+02, 2.75177221e+02,
       2.48874389e+02, 2.44152626e+02, 2.18953983e+02, 2.12301174e+02,
       2.02348752e+02, 1.87058454e+02, 1.85877551e+02, 1.57261155e+02,
       1.47719867e+02, 1.36235561e+02, 1.28273276e+02, 1.22289683e+02,
       1.17965373e+02, 8.92001814e+01, 8.48042427e+01, 7.68762316e+01,
      

In [11]:
np.max(np.linalg.norm(img, axis=0))

3768.126855614073

In [12]:
ΣB = (sum(Lambda) + len(Lambda) * 20) / (2 * 0.5) * np.eye(len(Lambda))

In [13]:
np.mean(np.linalg.norm(ΣB))

3220507.5829606145

In [14]:
empirical_cov = np.var(img, axis=(0, 3))

In [15]:
empirical_cov

array([[7037.31971728, 6907.88371728, 7080.88888889, ..., 7928.89207284,
        7963.10564321, 8066.39385062],
       [7071.87959877, 6847.89251358, 6983.75723951, ..., 7841.65751111,
        7834.80384691, 7807.0293284 ],
       [7016.53416667, 6914.55694321, 6919.8156284 , ..., 7612.70134691,
        7667.31755432, 7639.54560988],
       ...,
       [7142.99486914, 6763.4419    , 6521.67434444, ..., 7503.39718025,
        7602.0033321 , 7795.26016173],
       [6702.3335358 , 6406.01826667, 6422.8265321 , ..., 7352.94101111,
        7546.21728395, 7831.82760988],
       [6534.06666667, 6525.8476284 , 6541.75246914, ..., 7521.56540617,
        7467.32817778, 8000.25043333]])

In [16]:
c = 20
v = 0.5
beta = 0.5
sqrt_sum = sum(np.sqrt(Lambda[j] + 10 * c * v/ beta) for j in range(len(Lambda)))
Lambda_B = [2 * v / np.sqrt(Lambda[j] + 10 * c * v/ beta) * sqrt_sum for j in range(len(Lambda))]
ΛB = np.diag(Lambda_B)

        # Step 13: Determine the Gaussian noise covariance
ΣB = U @ np.linalg.inv(ΛB) @ UT

In [17]:
np.linalg.norm(ΣB)

0.24992504301452273

In [18]:
ΣB.shape

(64, 64)

In [19]:
np.min([np.abs(Lambda[j] - lambd)for j in range(j0 + 1) for lambd in Lambda])

0.0

In [ ]:
[np.abs(Lambda[j] - Lambda[l])for j in range(j0 + 1) for l in range(j0 + 1, len(Lambda))]

NameError: name 'lambd' is not defined